In [2]:
!pip install portalocker
!pip install torchmetrics

In [3]:
import argparse
import logging
import time

import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import DATASETS
from torchtext.prototype.transforms import load_sp_model, PRETRAINED_SP_MODEL, SentencePieceTokenizer
from torchtext.utils import download_from_url
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torchtext.vocab import GloVe
from tqdm import tqdm

torch.autograd.set_detect_anomaly(True)

FILL = '_FILL_'

### Information
- torchtext repo: https://github.com/pytorch/text/tree/main/torchtext
- torchtext documentation: https://pytorch.org/text/stable/index.html

### Constants

In [4]:
DATASET = "AG_NEWS"
DATA_DIR = ".data"
DEVICE = "cpu"
EMBED_DIM = 300
LR = 1.0
BATCH_SIZE = 128
NUM_EPOCHS = 5
PADDING_VALUE = 0
PADDING_IDX = PADDING_VALUE

### Get the tokenizer
- Use the WordLevel tokenizer.


In [5]:
# Get basic tokenizer
basic_english_tokenizer = get_tokenizer("basic_english")

In [6]:
basic_english_tokenizer("This is some text ...")

['this', 'is', 'some', 'text', '.', '.', '.']

In [7]:
# Needed later; set this to the tokenizer above
TOKENIZER = basic_english_tokenizer

### Get the data and get the vocabulary

In [8]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield TOKENIZER(text)

In [9]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
VOCAB = build_vocab_from_iterator(yield_tokens(train_iter), specials=('<pad>', '<unk>'))

# Make the default index the same as that of the unk_token.
VOCAB.set_default_index(VOCAB['<unk>'])

### Get GLOVE embeddings

In [10]:
# Get the GloVe embeddings; this will be slow
GLOVE = GloVe()

.vector_cache/glove.840B.300d.zip: 2.18GB [06:56, 5.22MB/s]                            
100%|█████████▉| 2196016/2196017 [03:58<00:00, 9210.06it/s]


In [11]:
len(GLOVE), GLOVE.vectors.shape

(2196017, torch.Size([2196017, 300]))

### Helper functions

In [12]:
def text_pipeline(text):
    return VOCAB(TOKENIZER(text))

def label_pipeline(label):
    return int(label) - 1

Nice link on collate_fn and DataLoader in PyTorch: https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

In [13]:
# As before, loop through the batch and transform into tensors
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        # Get the label from {1, 2, 3, 4} to {0, 1, 2, 3}
        label = label_pipeline(_label)
        label_list.append(label)

        # Return a list of ints
        processed_text = text_pipeline(_text)
        # Append to text_list
        text_list.append(torch.tensor(processed_text, dtype=torch.int64))

    # Pad and make into tensors as needed
    text_list = pad_sequence(text_list, batch_first=True, padding_value=PADDING_VALUE)
    label_list = torch.tensor(label_list, dtype=torch.int64)

    return label_list.to(DEVICE), text_list.to(DEVICE)

### Get the data

In [14]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
num_class = len(set([label for (label, _) in train_iter]))
# What are the classes?
print(f"The number of classes is {num_class} ...")

The number of classes is 4 ...


### Set up the model

Good reference on this type of model
- CNN for Sentence Classification: https://arxiv.org/pdf/1408.5882.pdf

You can build CNN models with either CNN1d or CNN2d.

In [15]:
class CNN1dTextClassificationModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        num_class,
        embed_dim = 300,
        use_pretrained = True,
        fine_tune_embeddings = True,
        debug = False
    ):

        super(CNN1dTextClassificationModel, self).__init__()

        self.embedding = nn.Embedding(
            vocab_size,
            embed_dim,
            padding_idx=PADDING_IDX
        )

        if use_pretrained:
            # Set the embeddings to not requiring gradients since we'll try and modify
            self.embedding.weight.requires_grad = fine_tune_embeddings
            for i in range(vocab_size):
                # Get the token for the index i
                token = VOCAB.lookup_token(i)
                # Modify the embedding for index i by the embedding for that token
                # Do this only if token is in the stoi dictionary for GLOVE
                if token in GLOVE.stoi:
                    self.embedding.weight.data[i] = GLOVE[token]
        else:
            # Otherwise, initialize the weights as specified below
            self.init_weights()

        # If weights do not get changed, turn off gradients for the GloVe embeddings
        if not fine_tune_embeddings:
            self.embedding.weight.requires_grad = False

        # Define a Conv1d layer that collapses all the channels and does not collapse the time dimension
        self.cnn1 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=1)

        # Define 3 Conv1d layers each having 1 filter and kernel sizes 2, 3 and 4
        self.cnn2 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=2)
        self.cnn3 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=3)
        self.cnn4 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=4)

        # A linear map from some dimensions to num_class (you need to figure it out)
        self.fc = nn.Linear(3, num_class)

        self.debug = debug

    def init_weights(self):
        initrange = 0.5
        # Initialize the embedding weight matrix to uniform between the [-0.5, 0.5]
        self.embedding.weight.data.uniform_(-initrange, initrange)
        # Initialize the weight matrix of fc to uniform between the [-0.5, 0.5]
        self.fc.weight.data.uniform_(-initrange, initrange)
        # Initialize the bias for fc to zero
        self.fc.bias.data.zero_()

    # B = batch_size, L = sequence length, D = vector dimension
    def forward(self, text):

        # B X L X D
        # Get the embeddings for the text passed in
        embedded = self.embedding(text)

        if self.debug:
            print('embedding', embedded.shape)

        # B X D X L
        # Transpose the embedding above as needed
        embedded = embedded.permute(0, 2, 1)

        # B X 1 X L
        # Pass through cnn1
        cnn1 = self.cnn1(embedded)
        if self.debug:
            print('cnn1', cnn1.shape)

        # B X 1
        # Take Max pooling over time
        cnn1 = F.max_pool1d(cnn1, cnn1.size(2)).squeeze(2)
        if self.debug:
          print('cnn1 after max pool', cnn1.shape)

        # B X 1 X L - 1
        # Pass through cnn2 and add a RELU
        cnn2 = F.relu(self.cnn2(embedded))
        if self.debug:
            print('cnn2', cnn2.shape)

        # B X 1 X L - 2
        # Pass through cnn3 and add a RELU
        cnn3 = F.relu(self.cnn3(embedded))
        if self.debug:
            print('cnn3', cnn3.shape)

        # B X 1 X L - 3
        # Pass through cnn4 and add a RELU
        cnn4 = F.relu(self.cnn4(embedded))
        if self.debug:
            print('cnn4', cnn4.shape)

        # B X 1 in all cases
        # Apply max pooling to each of cnn2, cnn3 and cnn4
        cnn2 = F.max_pool1d(cnn2, cnn2.size(2)).squeeze(2)
        cnn3 = F.max_pool1d(cnn3, cnn3.size(2)).squeeze(2)
        cnn4 = F.max_pool1d(cnn4, cnn4.size(2)).squeeze(2)

        # B X 1 in all cases
        # Apply max pooling over time
        if self.debug:
            print('cnn2 after max', cnn2.shape)

        # Add to each cnn2, 3, 4 a skip connection to cnn1 and average the results
        cnn2 = (cnn2 + cnn1) / 2
        cnn3 = (cnn3 + cnn1) / 2
        cnn4 = (cnn4 + cnn1) / 2
        if self.debug:
            print('cnn2 after skip connection', cnn2.shape)

        # B X 3
        # Concatenate the above
        cnn_concat = torch.cat((cnn2, cnn3, cnn4), 1)
        if self.debug:
            print('cnn concat', cnn_concat.shape)
            # Set the debug to False after the first forward pass
            self.debug = False
        # Pass through an appropriate Linear layer to get the right dimensions needed
        out = self.fc(cnn_concat)

        return out

### Set up the model

In [16]:
# If this is True, we will initialize the Embedding layer with GLOVE
USE_PRETRAINED = True,

# If this is True, we will allow for gradient updates on the nn.Embedding layer
FINE_TUNE_EMBEDDINGS = True

# Set the loss appropriately
criterion = nn.CrossEntropyLoss()

model = CNN1dTextClassificationModel(len(VOCAB), num_class, use_pretrained=USE_PRETRAINED, fine_tune_embeddings=FINE_TUNE_EMBEDDINGS)
# Set the optimizer to SGD
# Add an L2 regularizer of 0.00001
optimizer = torch.optim.SGD(model.parameters(), lr=LR, weight_decay=0.00001)

# Set the scheduler to StepLR with gamma=0.1 and step_size = 1.0
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

### Set up the data

In [17]:
train_iter, test_iter = DATASETS[DATASET]()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [18]:
iterator = iter(train_dataloader)

In [19]:
for label, text in iterator:
  print(label)
  print(text)
  break

tensor([2, 2, 1, 1, 0, 3, 2, 3, 0, 2, 1, 0, 1, 0, 3, 0, 3, 1, 1, 2, 3, 3, 3, 2,
        2, 0, 3, 3, 3, 2, 1, 1, 1, 1, 0, 3, 0, 2, 0, 3, 0, 3, 0, 0, 2, 1, 2, 1,
        1, 1, 2, 1, 1, 0, 0, 2, 3, 1, 0, 3, 3, 3, 3, 0, 3, 0, 2, 1, 0, 2, 0, 0,
        3, 3, 2, 2, 3, 1, 1, 2, 0, 2, 1, 2, 0, 1, 2, 1, 3, 0, 3, 1, 1, 1, 1, 2,
        0, 1, 1, 3, 3, 3, 2, 0, 2, 1, 2, 1, 2, 2, 1, 1, 0, 1, 3, 1, 3, 0, 3, 0,
        2, 3, 0, 1, 1, 3, 0, 0])
tensor([[   37,  2495,     5,  ...,     0,     0,     0],
        [    6,  6440,     7,  ...,     0,     0,     0],
        [  851,    89,   561,  ...,     0,     0,     0],
        ...,
        [   78, 11261,    64,  ...,     0,     0,     0],
        [  149,   511,   403,  ...,     0,     0,     0],
        [ 6444,  4103,  6401,  ...,     0,     0,     0]])


### Train the model

In [20]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_acc, total_count = 0, 0
    total_loss, total_batches = 0.0, 0
    total_zero_gradients_percentage = []
    log_interval = 10

    for idx, (label, text) in tqdm(enumerate(dataloader)):
        # Set gradients to zero
        optimizer.zero_grad()

        # Get the predictions
        predicted_label = model(text)
        # Get the loss
        loss = criterion(predicted_label, label)

        # Do back propagation and get the gradients
        loss.backward()

        # Get the loss per batch and the number of batches
        with torch.no_grad():
            total_loss += loss.item()
            total_batches += 1

        # Loop through all the parameters
        # Specifically, for this batch, get the percentage of zero gradients across all parameters
        # Append this to the list above which will print out the total every 10 batches
        total_nonzero_gradients = 0.0
        total_param_count = 0.0
        for param in model.parameters():
            total_nonzero_gradients += torch.sum(param.grad != 0)
            total_param_count += torch.numel(param.grad)

        # Append to total_zero_gradients_percentage
        zero_grad_percentage = (total_param_count - total_nonzero_gradients) / total_param_count
        total_zero_gradients_percentage.append(zero_grad_percentage)


        # Clip the gradient at ?? Should we use 10.0 or 0.1 with the learning rate we picked and the default notebook setting?
        # Use the above loop to help you figure this out
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)

        # Do an optimization step
        optimizer.step()

        # Get the accuracy
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

        # Log results
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f} "
                "| loss {:8.3f} "
                "| zero gradients percentage {:8.3f}".format(
                    epoch, idx,
                    len(dataloader),
                    total_acc / total_count,
                    total_loss / total_batches,
                    torch.tensor(total_zero_gradients_percentage).mean().item()
                    )
            )
            # Reset variables as needed
            total_acc, total_count = 0, 0
            total_loss, total_batches = 0.0, 0
            total_zero_gradients = []

In [21]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0.0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(input=predicted_label, target=label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()
    return total_acc / total_count, total_loss / total_count

In [22]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    accu_val, loss_val = evaluate(valid_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch,
            time.time() - epoch_start_time,
            accu_val,
            loss_val
            )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
accu_test, loss_test = evaluate(test_dataloader, model, criterion)
print("test accuracy {:8.3f}".format(accu_test))
print("test loss {:8.3f}".format(loss_test))

12it [00:02,  5.18it/s]

| epoch   1 |    10/  891 batches | accuracy    0.376 | loss    1.302 | zero gradients percentage    0.994


22it [00:04,  5.06it/s]

| epoch   1 |    20/  891 batches | accuracy    0.596 | loss    0.982 | zero gradients percentage    0.994


32it [00:06,  5.38it/s]

...

| epoch   5 |   870/  891 batches | accuracy    0.884 | loss    0.344 | zero gradients percentage    0.992


881it [03:00,  4.90it/s]

| epoch   5 |   880/  891 batches | accuracy    0.896 | loss    0.327 | zero gradients percentage    0.992


891it [03:02,  4.88it/s]


| epoch   5 |   890/  891 batches | accuracy    0.886 | loss    0.365 | zero gradients percentage    0.992
-----------------------------------------------------------
| end of epoch   5 | time: 183.78s | valid accuracy    0.875 
-----------------------------------------------------------
Checking the results of test dataset.
test accuracy    0.871
test loss    0.003
